In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [2]:
url = "https://www.flipkart.com/search?q=ac&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1"
response = requests.get(url)
response

<Response [200]>

In [8]:
Brand = [] 
Model_Year = [] 
Capacity = [] 
Energy_Rating = [] 
AC_Type = []
Feature = [] 
Prices = [] 
Ratings = [] 
No_of_Ratings = [] 
No_of_Reviews = []
Annual_Power_Usage = [] 
Room_Size = [] 
Warranty = []

headers = {"User-Agent": "Mozilla/5.0"}

for page_num in range(1, 50):  
    print(f"Scraping page {page_num}...")
    url = f"https://www.flipkart.com/search?q=ac&otracker=search&page={page_num}"
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    titles = soup.find_all("div", class_="KzDlHZ")
    price_tags = soup.find_all("div", class_="Nx9bqj _4b5DiR")
    rating_boxes = soup.find_all("div", class_="_5OesEi")
    details_blocks = soup.find_all("div", class_="_6NESgJ")

    for idx in range(len(titles)):
        a = titles[idx].text.strip()

        # Brand
        b = re.findall(r"^([A-Za-z]+)", a)
        Brand.append(b[0] if b else "N/A")

        # Model Year
        b = re.findall(r"(20\d{2})", a)
        Model_Year.append(b[0] if b else "N/A")

        # Capacity
        b = re.findall(r"(\d+(?:\.\d+)?)\s*Ton", a)
        Capacity.append(b[0] if b else "N/A")

        # Energy Rating
        b = re.findall(r"(\d)\s*Star", a)
        Energy_Rating.append(b[0] if b else "N/A")

        # AC Type
        b = re.findall(r"\b(Split|Window|Inverter)\b", a, re.IGNORECASE)
        AC_Type.append(b[0].capitalize() if b else "N/A")

        # Feature
        b = re.findall(r"([0-9]+-in-[0-9]+|[0-9]+ Step|[0-9]+-Way)", a, re.IGNORECASE)
        Feature.append(b[0] if b else "N/A")

        # Price
        Prices.append(price_tags[idx].text.strip().replace("₹", "").replace(",", "") if idx < len(price_tags) else "N/A")

        # Ratings & Reviews
        if idx < len(rating_boxes):
            rating_div = rating_boxes[idx]
            rating_value = rating_div.find("div", class_="XQDdHH")
            Ratings.append(rating_value.text.strip() if rating_value else "N/A")

            review_text = rating_div.find("span", class_="Wphh3N")
            if review_text:
                full_review_text = review_text.text.strip()
                r1 = re.findall(r"([\d,]+)\s+Ratings", full_review_text)
                r2 = re.findall(r"([\d,]+)\s+Reviews", full_review_text)
                No_of_Ratings.append(r1[0] if r1 else "N/A")
                No_of_Reviews.append(r2[0] if r2 else "N/A")
            else:
                No_of_Ratings.append("N/A")
                No_of_Reviews.append("N/A")
        else:
            Ratings.append("N/A")
            No_of_Ratings.append("N/A")
            No_of_Reviews.append("N/A")

        # Annual Power Usage, Room Size, Warranty
        if idx < len(details_blocks):
            detail_text = details_blocks[idx].text.strip()

            # Annual Power Usage
            apu = re.findall(r"([\d.]+)\s*Units", detail_text)
            Annual_Power_Usage.append(apu[0] if apu else "N/A")

            # Room Size
            rs = re.findall(r"(\d+\s*-\s*\d+\s*sqft|[\d]+\s*sqft\s*or\s*Below|Up to \d+\s*sqft)", detail_text, re.IGNORECASE)
            Room_Size.append(rs[0] if rs else "N/A")

            # Warranty (raw text)
            w = re.findall(r"(\d+\s*(?:Year|Years))", detail_text, re.IGNORECASE)
            Warranty.append(w[0] if w else "N/A")
        else:
            Annual_Power_Usage.append("N/A")
            Room_Size.append("N/A")
            Warranty.append("N/A")

    time.sleep(2)

# Create DataFrame
df = pd.DataFrame({
    "Brand": Brand,
    "Model_Year": Model_Year,
    "Capacity (Ton)": Capacity,
    "Energy_Rating (Star)": Energy_Rating,
    "AC_Type": AC_Type,
    "Feature": Feature,
    "Price (₹)": Prices,
    "Rating": Ratings,
    "No_of_Ratings": No_of_Ratings,
    "No_of_Reviews": No_of_Reviews,
    "Annual_Power_Usage (Units)": Annual_Power_Usage,
    "Room_Size": Room_Size,
    "Warranty(Years)": Warranty,
})

print("\n✅ Total Records:", len(df))
print(df.head(10))


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...

✅ Total Records: 984
     Br

In [14]:
# Save to CSV
df.to_csv("flipkart_ac_data.csv", index=False, encoding="utf-8-sig")
print("\n Data saved to flipkart_ac_data.csv")


 Data saved to flipkart_ac_data.csv


# Data Cleaning

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
# Load dataset
df = pd.read_csv("flipkart_ac_data.csv")

In [53]:
df.shape

(696, 15)

In [54]:
df.head(5)

,Brand,Model_Year,Capacity (Ton),Energy_Rating (Star),AC_Type,Feature,Price (₹),Rating,No_of_Ratings,No_of_Reviews,Annual_Power_Usage (Units),Room_Size,Warranty_Product (Years),Warranty_Component (Years),Warranty_Compressor (Years)
0,Voltas,2024.0,1.0,5.0,Split,NaN,30620,4.2,"1,792",233,511.13,90 sqft or Below,NaN,NaN,NaN
1,Samsung,2025.0,1.5,3.0,Split,5 Step,33590,4.1,"34,432","3,202",852.47,111 - 150 sqft,5.0,NaN,5.0
2,Godrej,2025.0,2.5,3.0,Split,5-In-1,48170,3.9,192,18,1330.81,NaN,5.0,NaN,5.0
3,Daikin,2024.0,1.5,3.0,Split,NaN,34590,4.2,"19,247","1,444",966.47,111 - 150 sqft,5.0,NaN,NaN
4,Daikin,2024.0,1.5,5.0,Split,NaN,42390,4.2,"13,441","1,191",785.67,111 - 150 sqft,5.0,NaN,NaN


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Brand                        696 non-null    object 
 1   Model_Year                   511 non-null    float64
 2   Capacity (Ton)               696 non-null    float64
 3   Energy_Rating (Star)         690 non-null    float64
 4   AC_Type                      696 non-null    object 
 5   Feature                      209 non-null    object 
 6   Price (₹)                    696 non-null    int64  
 7   Rating                       692 non-null    float64
 8   No_of_Ratings                692 non-null    object 
 9   No_of_Reviews                692 non-null    object 
 10  Annual_Power_Usage (Units)   615 non-null    float64
 11  Room_Size                    691 non-null    object 
 12  Warranty_Product (Years)     506 non-null    float64
 13  Warranty_Component (

In [56]:
# Remove commas and convert to integers
df['No_of_Ratings'] = df['No_of_Ratings'].str.replace(',', '', regex=True).astype(float)
df['No_of_Reviews'] = df['No_of_Reviews'].str.replace(',', '', regex=True).astype(float)

In [57]:
# missing values
print(df.isnull().sum())

Brand                            0
Model_Year                     185
Capacity (Ton)                   0
Energy_Rating (Star)             6
AC_Type                          0
Feature                        487
Price (₹)                        0
Rating                           4
No_of_Ratings                    4
No_of_Reviews                    4
Annual_Power_Usage (Units)      81
Room_Size                        5
Warranty_Product (Years)       190
Warranty_Component (Years)     644
Warranty_Compressor (Years)    166
dtype: int64


In [58]:
df['Model_Year'].fillna(df['Model_Year'].mode()[0], inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6712\3058788347.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Model_Year'].fillna(df['Model_Year'].mode()[0], inplace=True)


In [59]:
df['Energy_Rating (Star)'].fillna(df['Energy_Rating (Star)'].median(), inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6712\168339988.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Energy_Rating (Star)'].fillna(df['Energy_Rating (Star)'].median(), inplace=True)


In [60]:
df['Feature'].fillna("Standard", inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6712\3060322206.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Feature'].fillna("Standard", inplace=True)


In [61]:
df['Rating'].fillna(df['Rating'].mean(), inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6712\1501272357.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Rating'].fillna(df['Rating'].mean(), inplace=True)


In [62]:
df['Annual_Power_Usage (Units)'].fillna(df['Annual_Power_Usage (Units)'].median(), inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6712\2544732012.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Annual_Power_Usage (Units)'].fillna(df['Annual_Power_Usage (Units)'].median(), inplace=True)


In [63]:
df['Room_Size'].fillna("Unknown", inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6712\2815538612.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Room_Size'].fillna("Unknown", inplace=True)


In [65]:
df['Model_Year'] = df['Model_Year'].astype(int)

In [66]:
# Fill missing values and convert to int using NumPy
df['Model_Year'] = np.where(df['Model_Year'].isna(),
                            df['Model_Year'].mode()[0],
                            df['Model_Year']).astype(int)

df['Warranty_Product (Years)'] = np.where(df['Warranty_Product (Years)'].isna(),
                                          1,
                                          df['Warranty_Product (Years)']).astype(int)

df['Warranty_Component (Years)'] = np.where(df['Warranty_Component (Years)'].isna(),
                                            0,
                                            df['Warranty_Component (Years)']).astype(int)

df['Warranty_Compressor (Years)'] = np.where(df['Warranty_Compressor (Years)'].isna(),
                                             5,
                                             df['Warranty_Compressor (Years)']).astype(int)

In [67]:
df.to_csv("cleaned_ac_data.csv", index=False)

In [68]:
df1=pd.read_csv("cleaned_ac_data.csv")

In [69]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Brand                        696 non-null    object 
 1   Model_Year                   696 non-null    int64  
 2   Capacity (Ton)               696 non-null    float64
 3   Energy_Rating (Star)         696 non-null    float64
 4   AC_Type                      696 non-null    object 
 5   Feature                      696 non-null    object 
 6   Price (₹)                    696 non-null    int64  
 7   Rating                       696 non-null    float64
 8   No_of_Ratings                692 non-null    float64
 9   No_of_Reviews                692 non-null    float64
 10  Annual_Power_Usage (Units)   696 non-null    float64
 11  Room_Size                    696 non-null    object 
 12  Warranty_Product (Years)     696 non-null    int64  
 13  Warranty_Component (

In [75]:
df.head()

,Brand,Model_Year,Capacity (Ton),Energy_Rating (Star),AC_Type,Feature,Price (₹),Rating,No_of_Ratings,No_of_Reviews,Annual_Power_Usage (Units),Room_Size,Warranty_Product (Years),Warranty_Component (Years),Warranty_Compressor (Years)
0,Voltas,2024,1.0,5.0,Split,Standard,30620,4.2,1792.0,233.0,511.13,90 sqft or Below,1,0,5
1,Samsung,2025,1.5,3.0,Split,5 Step,33590,4.1,34432.0,3202.0,852.47,111 - 150 sqft,5,0,5
2,Godrej,2025,2.5,3.0,Split,5-In-1,48170,3.9,192.0,18.0,1330.81,Unknown,5,0,5
3,Daikin,2024,1.5,3.0,Split,Standard,34590,4.2,19247.0,1444.0,966.47,111 - 150 sqft,5,0,5
4,Daikin,2024,1.5,5.0,Split,Standard,42390,4.2,13441.0,1191.0,785.67,111 - 150 sqft,5,0,5
